# Lezione 4: Teoria della Quantizzazione

**Obiettivo:** Comprendere le tecniche di quantizzazione lineare, implementare la quantizzazione dei pesi, strategie di calibrazione, link a  SOTA papers.


# 1. Introduzione alla Quantizzazione

La quantizzazione è il processo di riduzione della precisione di valori, in genere da un formato di maggiore precisione a uno di minore precisione, che riduce la dimensione del modello e accelera l'inferenza, generalmente con degradazione minima dell'accuratezza.

**Perché Quantizzare?**
- **Risparmio di memoria:** 32-bit → 8-bit comporta una riduzione di 4× nella dimensione del modello.
- **Efficienza di calcolo:** Operazioni intere (INT8) sono più veloci e più energy-efficient su molti acceleratori.
- **Deployment:** Modelli più piccoli si adattano a dispositivi edge e riducono i requisiti di banda.

**Panoramica della Quantizzazione Lineare:**
1. **Stima dell'intervallo:** trova $r_{\min}$ e $r_{\max}$ del tensore.
2. **Calcolo dei parametri:**  
   $$s = \frac{r_{\max} - r_{\min}}{q_{\max} - q_{\min}}, \quad z = \mathrm{round}\bigl(-r_{\min} / s\bigr).$$
3. **Quantizza:**  
   $$q = \mathrm{clip}(\mathrm{round}(r / s) + z, \; q_{\min}, q_{\max}).$$
4. **Dequantizza:**  
   $$\hat r = s\,(q - z).$$
- Qui, $[q_{\min}, q_{\max}]$ è tipicamente $[-128, 127]$ per INT8 firmato.
- L'**errore di quantizzazione** $\hat r - r$ può essere mitigato con calibrazione e training.


In [1]:
# Funzione per calcolare scala (s) and zero-point (z)
def get_quant_params(r_min, r_max, q_min=-128, q_max=127):
    """
    Returns scale s and zero-point z for mapping floats in [r_min, r_max]
    to ints in [q_min, q_max].
    """
    s = (r_max - r_min) / (q_max - q_min)
    z = int(round(-r_min / s))
    z = max(q_min, min(q_max, z))
    return s, z

# Esempio
r_min, r_max = -1.0, 1.0
scale, zero_point = get_quant_params(r_min, r_max)
print(f"Scale: {scale:.4f}, Zero-point: {zero_point}")

Scale: 0.0078, Zero-point: 127


In [2]:
import numpy as np

def quantize_tensor(r, s, z, q_min=-128, q_max=127):
    q = np.round(r / s) + z
    return np.clip(q, q_min, q_max).astype(np.int8)

def dequantize_tensor(q, s, z):
    return s * (q.astype(np.int32) - z)

# Synthetic example
tensor = np.linspace(-1, 1, num=10)
q_tensor = quantize_tensor(tensor, scale, zero_point)
reconstructed = dequantize_tensor(q_tensor, scale, zero_point)

print("Originale:", tensor)
print("Quantizzato:", q_tensor)
print("Riconstruito:", np.round(reconstructed, 4))
print("Errore di ricostruzione:", np.round(reconstructed - tensor, 4))

Originale: [-1.         -0.77777778 -0.55555556 -0.33333333 -0.11111111  0.11111111
  0.33333333  0.55555556  0.77777778  1.        ]
Quantizzato: [ -1  28  56  84 113 127 127 127 127 127]
Riconstruito: [-1.0039 -0.7765 -0.5569 -0.3373 -0.1098  0.      0.      0.      0.
  0.    ]
Errore di ricostruzione: [-0.0039  0.0013 -0.0013 -0.0039  0.0013 -0.1111 -0.3333 -0.5556 -0.7778
 -1.    ]


## 0. Configurazione


In [3]:
# Installa le librerie richieste
#!/pip install transformers==4.35.0
#!/pip install quanto==0.0.11
#!/pip install torch==2.1.1
#!/pip install datasets


In [4]:
# Import e setup degli helper
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from helper import compute_module_sizes
import numpy as np
from datasets import load_dataset


## 1. Carica modello


In [5]:
# Carica modello e tokenizer
model_name = "EleutherAI/pythia-410m"
model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Calcola la dimensione del modello FP32 
total_bytes = sum(compute_module_sizes(model).values())
print(f"Dimensione del modello FP32: {total_bytes / 1024**3:.2f} GB")

Dimensione del modello FP32: 9.23 GB


## 2. Quantizzazione dei pesi con `quanto`


In [6]:
from quanto import quantize, freeze

# Applica quantizzazione statica INT8 ai pesi
quantize(model, weights=torch.int8, activations=None)

# Finalizza il modello quantizzato (freeze modifica il modello in-place)
freeze(model)
qmodel = model

# Calcola la dimensione del modello quantizzato
total_q_bytes = sum(compute_module_sizes(qmodel).values())
print(f"Dimensione del modello INT8: {total_q_bytes / 1024**3:.2f} GB")

Dimensione del modello INT8: 3.22 GB


In [7]:
# Prova con un modello più piccolo come nel tutorial L4: FLAN-T5 small

#!pip install sentencepiece==0.2.0

In [8]:
# Prova con un modello più piccolo come nel tutorial L4: FLAN-T5 small

# Carica Google FLAN-T5 Small 
import sentencepiece as spm 
from transformers import T5Tokenizer, T5ForConditionalGeneration
from helper import compute_module_sizes

model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Genera un esempio di output
input_text = "Hello, my name is "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print("FP32 output:", tokenizer.decode(outputs[0], skip_special_tokens=True))

# Calcola la dimensione del modello FP32 
module_sizes = compute_module_sizes(model)
print(f"Dimensione del modello FP32: {module_sizes[''] / 1024**3:.2f} GB")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


FP32 output: annie scott
Dimensione del modello FP32: 0.29 GB


In [9]:
# Quantizza i pesi di Google FLAN-T5 small a INT8
from quanto import quantize, freeze
import torch

quantize(model, weights=torch.int8, activations=None)

# Mostra il modello per confermare quantization wrappers
print(model)

# Finalizza il modello quantizzato (freeze modifica il modello in-place)
freeze(model)
qmodel = model

# Calcola la dimensione del modello quantizzato
total_q_bytes = sum(compute_module_sizes(qmodel).values())
print(f"Dimensione del modello INT8: {total_q_bytes / 1024**3:.2f} GB")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): QLinear(in_features=512, out_features=384, bias=False)
              (k): QLinear(in_features=512, out_features=384, bias=False)
              (v): QLinear(in_features=512, out_features=384, bias=False)
              (o): QLinear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): QLinear(in_features=512, out_features=1024, bias=False)
              (wi_1): QLinear(in_features=512, out_features=1024, bias=False)
              

In [10]:
# Confronta gli output di generazione di testo

def sample_text(mdl, prompt="Hello, my name is", max_new_tokens=10):
    inputs = tokenizer(prompt, return_tensors="pt")
    out = mdl.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(out[0], skip_special_tokens=True)

print("FP32 ->", sample_text(model))
print("INT8 ->", sample_text(qmodel))

FP32 -> annie scott
INT8 -> annie scott


## 3. Quantizzazione delle Attivazioni e Calibrazione


In [11]:
# La quantizzazione delle attivazioni richiede moduli corrispondenti (es. nn.Linear).
# FLAN-T5 usa blocchi personalizzati; le attivazioni potrebbero non essere quantizzate di default.

from transformers import T5ForConditionalGeneration
# Re-load del modello originale in FP32 
model_act = T5ForConditionalGeneration.from_pretrained(model_name)

# Quantizza sia i pesi che le attivazioni 
quantize(model_act, weights=torch.int8, activations=torch.int8)

# Verifica quali moduli
from quanto.nn.qlinear import QLinear
wrapped = [(n, m) for n, m in model_act.named_modules() if isinstance(m, QLinear)]
if not wrapped:
    print("No QLinear modules found. Weight+activation quantization skipped for custom layers.")
else:
    print(f"Found {len(wrapped)} QLinear modules for activation quantization.")

# Prepara texts e batch size per la calibratione
texts = load_dataset("wikitext", "wikitext-2-raw-v1", split="validation")["text"][:50]
batch_size = 8

# Calibrazione
# Useremo input_ids come decoder_input_ids per soffisfare i forward requirements
model_act.eval()
with torch.no_grad():
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True)
        # Provide decoder_input_ids equal to input_ids for teacher forcing
        batch_outputs = model_act(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            decoder_input_ids=inputs.input_ids
        )

# Freeze il modello calibrato 
freeze(model_act)
qmodel_act = model_act

# Calcola la nuova dimensione del modello e generazione di un esempio
size_act = sum(compute_module_sizes(qmodel_act).values()) / 1024**3
print(f"Dimensione del modello calibrato INT8: {size_act:.2f} GB")
# Usa generate per produrre testo
# print("Generazione di esempio dopo la quantizzazione delle attivazioni:", sample_text(qmodel_act))

# Nota:
# - Abbiamo passato solo `decoder_input_ids` per la calibrazione, il che potrebbe non catturare completamente la dinamica del decoder durante la generazione del testo.
# - La calibrazione su output teacher-forced può lasciare il modello impreparato per la generazione libera, portando a risultati privi di senso.



Found 145 QLinear modules for activation quantization.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Dimensione del modello calibrato INT8: 0.66 GB


## 4. Quantization-Aware Training (QAT)
Il quantization-aware training intreccia la quantizzazione durante il training in modo che il modello impari ad adattarsi alla precisione inferiore.

**Concetto chiave:** Mantenere due set di pesi:
- **Pesi float:** usati per gli aggiornamenti dei gradienti.
- **Pesi fake-quantizzati:** usati nel forward per simulare il comportamento INT8.

Durante il backprop, i gradienti scorrono attraverso la fake quantization ("Straight-Through Estimator") per aggiornare i pesi float.

**Fasi principali:**
1. Inserire wrapper di quant/dequant intorno ai layer chiave.
2. Eseguire il training; il forward usa pesi quantizzati, il backward aggiorna pesi float.
3. Esportare i pesi finali quantizzandoli.


## 5. Quantizzazione SOTA recente in LLMs

| Method   | Bits | Key Idea                                    | Calibration? | Summary                                                                                         | Paper Link                                      |
|----------|------|---------------------------------------------|--------------|-------------------------------------------------------------------------------------------------|--------------------------------------------------|
| LLM.INT8 | 8    | Outlier-aware two-stage (detect & scale)    | No           | Detects and rescales rare large weights (outliers) to minimize distortion.                       | [LLM.INT8](https://arxiv.org/abs/2208.07339)      |
| QLoRA    | 4    | LoRA adapters + QAT at 4-bit                | No           | Fine-tunes low-rank adapters on a quantized model to recover performance in 4-bit precision.      | [QLoRA](https://arxiv.org/abs/2305.14314)        |
| AWQ      | 4    | Per-channel activation-aware scaling        | Yes          | Learns per-channel scales based on activation statistics, improving INT4/INT8 accuracy.          | [AWQ](https://arxiv.org/abs/2306.00978)         |
| GPTQ     | 4    | Hessian-aware greedy rounding               | No           | Uses Hessian information to guide greedy weight rounding, preserving model loss surface.          | [GPTQ](https://arxiv.org/abs/2210.17323)     |
| HQQ      | 2    | Hybrid quant + learned reconstruction       | Yes          | Combines coarse quantization with reconstruction layers to achieve robust 2-bit representation. | [HQQ](https://mobiusml.github.io/hqq_blog/)         |
| QuIP     | 2    | Importance-based pruning + quantization     | Yes          | Prunes negligible weights based on importance, then applies quantization to remaining ones.       | [QuIP](https://arxiv.org/abs/2307.13304)        |


> **Suggerimento:** Molti di questi metodi includono implementazioni open-source; sperimenta con modelli piccoli per familiarizzare con le pipeline.


## 6. Conclusioni & Riferimenti

**Punti chiave:**
- La quantizzazione lineare (FP32→INT8) offre vantaggi significativi in memoria e calcolo con formule semplici.
- `quanto` automatizza la quantizzazione dei pesi; la calibrazione affina ulteriormente le attivazioni.
- Il QAT integra la quantizzazione nel training per ridurre l'errore.
- La quantizzazione SOTA per LLM varia da 8-bit a 2-bit, bilanciando dimensione e accuratezza.

**Riferimenti:**
1. T. Dettmers, "LLM.INT8: Outlier-aware Quantization" (Aug 2022)
2. T. Dettmers, "QLoRA: 4-bit LoRA Fine-tuning" (May 2023)
3. R. Fan et al., "AWQ: Activation-Aware Quantization" (2024)
4. A. Frantar et al., "GPTQ: Optimal Brain Quantization" (2023)
5. H. Badri et al., "HQQ: Hybrid Quantization with Reconstruction" (Nov 2023)
6. R. Tseng et al., "QuIP: Quantization via Importance-based Pruning" (Jul 2023)
